In [ ]:
import re

with open('input.txt') as f:
    data = f.read().splitlines()

valves = {}

# Create dictionnary of valves
for line in data:
    # regex numbers
    flow_rate = re.findall(r'\d+', line)[0]
    # regex double capital letters
    capital_letters = re.findall(r'[A-Z][A-Z]', line)
    valve_name = capital_letters[0]
    valve_outputs = capital_letters[1:]

    valves[valve_name] = {'flow_rate': int(flow_rate), 'outputs': valve_outputs}

# Create dictionnary of paths
paths = {}
timer = 30
loop_count = 0

def explore(valve_name, timer, pressure):
    global loop_count
    loop_count += 1
    # if loop_count > 2000:
    #     return
    path = {}
    timer = timer - 1
    for output_valve in valves[valve_name]['outputs'] :
        if timer > 0:
            path[output_valve] = {
                'pressure': int(pressure + valves[output_valve]['flow_rate'] * timer),
                'timer': timer,
                'path' : explore(output_valve, timer, pressure)
                }
        else:
            continue
    return path

paths = {'pressure': 0, 'timer': timer, 'path' : explore('AA', timer, 0)}

print(paths)

I had to watch a solution to make this one.

Thanks a lot to 0xdf for explaining his solution in [this video](https://www.youtube.com/watch?v=rN4tVLnkgJU).

I then can use cache trick with frozenset.

In [40]:
import re
from functools import cache

with open('input.txt') as f:
    data = f.read().splitlines()

valves = {}
opened = frozenset()

# Create dictionnary of valves
for line in data:
    # regex numbers
    flow_rate = re.findall(r'\d+', line)[0]
    # regex double capital letters
    capital_letters = re.findall(r'[A-Z][A-Z]', line)
    valve_name = capital_letters[0]
    valve_outputs = capital_letters[1:]

    valves[valve_name] = {'flow_rate': int(flow_rate), 'outputs': valve_outputs}

@cache
def resolve(timer, opened, valve_name, elefant_waits):
    max_score = 0
    # For a defined time, we we'll decide if we prefer turning the valve and explore or just explore
    if timer > 0:
        # Compute the exploration possible values
        explore_score = [resolve(timer - 1, opened, output, elefant_waits) for output in valves[valve_name]['outputs']]
        max_explore_score = max(explore_score)
        max_score = max_explore_score

        # Compute turning the valve (takes one minute) then explore
        if valves[valve_name]['flow_rate'] > 0 and valve_name not in opened:
            opened_set = set(opened)
            opened_set.add(valve_name)
            opened_frozenset = frozenset(opened_set)
            turn_score = [(valves[valve_name]['flow_rate'] * (timer - 1)) + resolve(timer - 2, opened_frozenset, output, elefant_waits) for output in valves[valve_name]['outputs']]
            max_turn_score = max(turn_score)

            if max_turn_score > max_explore_score:
                max_score = max_turn_score
    # If the human finished its score, we'll run another round with the elefant
    elif elefant_waits:
        return resolve(26, opened, 'AA', False)
    return max_score

print(f"Part 1 answer : {resolve(30, opened, 'AA', False)}")
print(f"Part 2 answer : {resolve(26, opened, 'AA', True)}")

Part 1 answer : 2077
Part 2 answer : 2741
